Simplified vorcity for transport equation for Stokes flow:

$$\nabla^2\omega = 0\quad\quad(1)$$

stream  function canbe defined:

$$u = \dfrac{\partial \psi}{\partial y}~and~v = -\dfrac{\partial \psi}{\partial x}\quad\quad(2)$$

In 2d, Vorticity should be:

$$\omega = \dfrac{\partial v}{\partial x}-\dfrac{\partial u}{\partial y}\quad\quad(3)$$

substituting $u,v$ showed in Eq(1) into Eq(2):

$$\nabla^2\psi = -\omega\quad\quad(4)$$

Above Eq (4) combined with Eq(1):

$$\nabla^4\psi = 0\quad\quad(5)$$

Given boundary condition:

At $x = 0$:

$$\psi = 0,~\dfrac{\partial \psi}{\partial x} = 0$$

At $x = 1$:

$$\psi = 0,~\dfrac{\partial \psi}{\partial x} = 0$$

At $y = 0$:

$$\psi = 0,~\dfrac{\partial \psi}{\partial y} = 0$$

At $y = 1$:

$$\psi = 0,~\dfrac{\partial \psi}{\partial y} = 1$$

In [1]:
import numpy
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [2]:
def plot_3D(x, y, p):

    fig = pyplot.figure(figsize=(11,7), dpi=100)
    ax = fig.gca(projection='3d')
    X,Y = numpy.meshgrid(x,y)
    surf = ax.plot_surface(X,Y,p[:], rstride=1, cstride=1, cmap=cm.viridis,
            linewidth=0, antialiased=False)

    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.view_init(30,45)

In [3]:
nx = 41
ny = 41

l = 1.
h = 1.

dx = l/(nx-1)
dy = h/(ny-1)

l1_target = 1e-6

##initial conditions
p_i = numpy.zeros((ny,nx)) ##create a XxY vector of 0's
psi_i = numpy.zeros((ny,nx)) ##create a XxY vector of 0's

##plotting aids
x = numpy.linspace(0.0,l,nx)
y = numpy.linspace(0.0,h,ny)

X,Y = numpy.meshgrid(x,y)

In [4]:
def L1norm(new, old):
    norm = numpy.sum(numpy.abs(new-old))
    return norm

With given condition why not solve it with:

$$\nabla^2 \omega = 0-laplace~equation$$
    
$$\nabla^2 \psi = -\omega-poisson~eqaution$$

In [5]:
def lap_poi(p,psi,dx,dy,l1_target):
    
    l1norm = 1
    l2norm = 1
    pn = numpy.empty_like(p)
    psi_n = numpy.empty_like(psi)
    iterations = 0
    
    while l1norm > l1_target or l2norm > l1_target:
    
        pn = p.copy()
        psi_n = psi.copy()
    
    
        psi[1:-1,1:-1] = ( (psi_n[1:-1,:-2] + psi_n[1:-1,2:])*dy**2 +\
                             (psi_n[:-2,1:-1] + psi_n[2:,1:-1])*dx**2 +\
                            p[1:-1,1:-1]*dx**2*dy**2 )\
                        /(2*(dx**2+dy**2))
        
        p[1:-1,1:-1] = .25*(pn[1:-1,2:] + pn[1:-1, :-2] +\
                        pn[2:, 1:-1] + pn[:-2, 1:-1])
        
        
        #Boundary condition for omega and psi:
        #top surface:
        p[-1,1:-1] = -1/(2*dy**2)*(8*psi[-2,1:-1]-psi[-3,1:-1]) - 3/dy
        #psi[-1,1:-1] = 0 # this term stands for O(delta y^2)
        #psi[-1,1:-1] = psi[-2, 1:-1] + dy
        
        #bottom surface:
        p[0,1:-1] = -1/(2*dy**2)*(8*psi[1,1:-1]-psi[2,1:-1])
        #psi[0,:] = 0
        #psi[0,1:-1] = psi[1,1:-1]
        
        #left surface:
        p[1:-1,0]=-1/(2*dy**2)*(8*psi[1:-1,1]-psi[1:-1,2])
        #psi[1:-1,0] = 0
        #psi[1:-1,0] = psi[1:-1,1]
        
        #right surface:
        p[1:-1,-1]=-1/(2*dy**2)*(8*psi[1:-1,-2]-psi[1:-1,-3])
        #psi[1:-1,-1] = 0
        #psi[1:-1,-1]=psi[1:-1,-2]
        
        l1norm = L1norm(p,pn)
        l2norm = L1norm(psi,psi_n)
    
    print(l1norm)
    print(l2norm)
    return p, psi

In [6]:
p,psi = lap_poi(p_i.copy(),psi_i.copy(),dx,dy,l1_target)
print()
print('The maximum value for psi is {:.4f}'.format(numpy.max(numpy.abs(psi))) )
print()
print('The maximum value for omega is {:.4f}'.format(numpy.max(numpy.abs(p))) )


9.99748384019e-07
2.24232961693e-08

The maximum value for psi is 0.0998

The maximum value for omega is 78.1927


In [7]:
a = numpy.round(psi[32,::8],4)
for i in range(len(a)):
    print('The {} number is: {:.4f}'.format(i+1,a[i]))

The 1 number is: 0.0000
The 2 number is: -0.0575
The 3 number is: -0.0944
The 4 number is: -0.0944
The 5 number is: -0.0575
The 6 number is: 0.0000


# Question:

If added all Neumann condition to solve this problem, the answer is completely different. However, $\psi$ should satisfy all eight boundary conditions, why neumann conditions neglected here? 